<a href="https://colab.research.google.com/github/sadrabr/LLM_projects/blob/main/Create_Brochure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
# Install requirement
!pip install python-dotenv

In [72]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [73]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [74]:
# Start Ollama server

import subprocess

# Start the command in a background process
process = subprocess.Popen(["ollama", "serve"])

# The kernel can continue execution while the process runs in the background
print("The 'ollama serve' process is running in the background.")

The 'ollama serve' process is running in the background.


In [96]:
# Download the model that we want to use - this might take a couple of mins
# The exclamation mark at the start of the line indicates that this is a shell command, not python

!ollama pull llama3.2:1b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling 74701a8c35f6: 100% ▕▏ 1.3 GB                         
pulling 966de95ca8a6: 100% ▕▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕▏ 6.0 KB                         
pulling 4f659a1e86d7: 100% ▕▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [97]:
# install the Ollama python API package

!pip install ollama

In [77]:
# import the package

import ollama

In [78]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [79]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-

In [80]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON format only as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [81]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON format only as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [82]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL  in JSON format only. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [83]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL  in JSON format only. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/01

In [84]:
import json
def get_links(url):
  website = Website(url)
  response = ollama.chat(
      model = "llama3.2:1b",
     messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
      # format={"type": "json_object"}
  )
  result = response['message']['content']
  return json.loads(result)

In [85]:
huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/nvidia/parakeet-tdt-0.6b-v2',
 '/nari-labs/Dia-1.6B',
 '/ACE-Step/ACE-Step-v1-3.5B',
 '/Lightricks/LTX-Video',
 '/deepseek-ai/DeepSeek-Prover-V2-671B',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/RiverZ/ICEdit',
 '/spaces/smolagents/computer-agent',
 '/spaces/nari-labs/Dia-1.6B',
 '/spaces/ACE-Step/ACE-Step',
 '/spaces',
 '/datasets/nvidia/Nemotron-CrossThink',
 '/datasets/DMindAI/DMind_Benchmark',
 '/datasets/nvidia/OpenMathReasoning',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transfor

In [86]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'},
  {'type': 'enterprise', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'docs', 'url': 'https://huggingface.co/docs'}]}

In [87]:
get_links("http://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}]}

## Second step: make the brochure!

In [88]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [89]:
get_all_details("https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'enterprise', 'url': 'https://huggingface.co/enterprise'}]}


"Landing page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nnvidia/parakeet-tdt-0.6b-v2\nUpdated\n8 days ago\n•\n42.1k\n•\n632\nnari-labs/Dia-1.6B\nUpdated\n4 days ago\n•\n144k\n•\n1.99k\nACE-Step/ACE-Step-v1-3.5B\nUpdated\n3 days ago\n•\n267\nLightricks/LTX-Video\nUpdated\n3 days ago\n•\n203k\n•\n1.33k\ndeepseek-ai/DeepSeek-Prover-V2-671B\nUpdated\n9 days ago\n•\n6.77k\n•\n745\nBrowse 1M+ models\nSpaces\nRunning\n6.23k\n6.23k\nDeepSite\n🐳\nGenerate any application with DeepSeek\nRunning\non\nZero\n427\n427\nICEdit\n🖼\nUniversal Image Editing is worth a single LoRA\nRunning\non\nCPU Upgrade\n292\n292\nComputer Agent\n🖥\nControl a virtual comp

In [90]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

In [91]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [92]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [93]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'enterprise', 'url': 'https://ui.endpoints.huggingface.co'}, {'type': 'docs', 'url': 'https://huggingface.co/docs'}, {'type': 'brand', 'url': 'https://github.com/huggingface'}, {'type': 'terms-of-service', 'url': 'https://apply.workable.com/huggingface/'}]}


"You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nnvidia/parakeet-tdt-0.6b-v2\nUpdated\n8 days ago\n•\n42.1k\n•\n632\nnari-labs/Dia-1.6B\nUpdated\n4 days ago\n•\n144k\n•\n1.99k\nACE-Step/ACE-Step-v1-3.5B\nUpdated\n3 days ago\n•\n267\nLightricks/LTX-Video\nUpdated\n3 days ago\n•\n203k\n•\n1.33k\ndeepseek-ai/DeepSeek-Prover-V2-671B\nUpdated\n9 days ago\n•\n6.77k\n•\n745\nBrowse 1M+ models\nSpaces\nRunning\n6.23k\n6.23k\nDeepSite\n🐳\nGenerate an

In [94]:
def create_brochure(company_name, url):
    response = ollama.chat(
      model = "llama3.2:1b",
     messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
      ])
    result = response['message']['content']
    display(Markdown(result))

In [98]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'enterprise', 'url': 'https://huggingface.co/enterprise'}, {'type': 'docs', 'url': 'https://huggingface.co/docs'}, {'type': 'brand', 'url': 'https://huggingface.co/brand'}]}


# Hugging Face: The AI Community Building The Future

## Who We Are

Hugging Face is a community-driven platform for AI and machine learning. Our mission is to provide a collaborative space where researchers, developers, and enthusiasts can come together to share models, datasets, and applications.

### Models

We have a vast collection of high-quality models available on our platform, including:

* **NVIDIA/Parakeet-tdt-0.6b-v2**: A state-of-the-art language model trained on the Parakeet dataset.
* **nari-labs/Dia-1.6B**: A large-scale language model that generates human-like text.

### Datasets

Our platform provides access to a vast library of datasets, including:

* **NVIDIA/Nemotron-CrossThink**: A benchmark dataset for neural machine translation tasks.
* **DMindAI/DMind_Benchmark**: A benchmark dataset for mathematical reasoning tasks.

## Spaces

We offer a range of spaces where you can host and collaborate on unlimited public models, datasets, and applications. Some popular spaces include:

* **DeepSite**: Generate any application with DeepSeek using our universal image editing tool.
* **ICEdit**: Universal image editing is worth a single LoRA (lightfield array) update.

## Pricing

Our platform offers various pricing plans for different use cases:

* **Compute**: Deploy on optimized inference endpoints or update your spaces applications to a GPU in a few clicks. Starting at $0.60/hour for GPU.
* **Enterprise**: Give your team the most advanced platform to build AI with enterprise-grade security, access controls, and dedicated support. Starting at $20/user/month.

## Jobs

We're always looking for talented individuals to join our team! Check out our job openings:

* [Press](https://huggingface.co/press): Explore all modalities (text, image, video, audio or 3D) and build your portfolio.
* [Compute](https://huggingface.co/compute): Deploy on optimized inference endpoints or update your spaces applications to a GPU in a few clicks.

## Resources

Stay up-to-date with the latest news and updates from Hugging Face:

* [Press](https://huggingface.co/press)
* [Resources](https://huggingface.co/resources)

## Learn

Check out our documentation, blog, forum, and social media channels to learn more about AI and machine learning:

* [Documentation](https://huggingface.co/docs)
* [Blog](https://blog.huggingface.com)
* [Forum](https://forum.huggingface.com)
* [Social](https://social.huggingface.com)

## About

Want to know more about Hugging Face? Here's a brief overview of our company:

### About (Sergei)

I'm the founder and CEO of Hugging Face. I'm passionate about AI and machine learning, and I believe that everyone should have access to this technology.

### Follow

Follow us on social media to stay up-to-date with the latest news and updates from Hugging Face:

* [Twitter](https://twitter.com/huggingface)
* [LinkedIn](https://linkedin.com/company/hugging-face/)
* [Discord](https://discord.com/invite/hugging-face)

## Terms of Service

Please read our terms of service before using our platform:

[Terms of Service](https://huggingface.co/terms-of-service)

In [99]:
def stream_brochure(company_name, url):
    stream =  ollama.chat(
      model = "llama3.2:1b",
     messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
      ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        # بررسی اینکه chunk اصلاً content داره یا نه
        content_piece = getattr(chunk, 'message', {}).get('content', '')
        response += content_piece
        response = response.replace("```", "").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [100]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'enterprise', 'url': 'https://www.huggingface.co/enterprise'}, {'type': 'pricing', 'url': 'https://ui.endpoints.huggingface.co/pricing'}, {'type': 'docs', 'url': 'https://github.com/huggingface'}, {'type': 'docs/blog', 'url': 'https://discuss.huggingface.co'}, {'type': 'docs/trl', 'url': 'https://www.zhihu.com/org/huggingface'}]}


## Hugging Face: The AI Community Building the Future

**The Platform Where Machine Learning Meets Collaboration**

Hugging Face is a community-driven platform built on machine learning, where researchers, developers, and businesses come together to share models, datasets, and applications. With over 1 million public models available, 250k+ datasets, and 50k+ organizations using Hugging Face AI, we're making it easy for everyone to accelerate their machine learning journey.

### Models

Our platform offers a wide range of models, including:

* **State-of-the-art transformers**: State-of-the-art ML for PyTorch, TensorFlow, JAX
* **Diffusers**: State-of-the-art Diffusion models in PyTorch
* **Safetensors**: Safe way to store/distribute neural network weights
* **Transformers.js**: State-of-the-art ML running directly in your browser

### Datasets

We provide access to a vast collection of datasets, including:

* **OpenMathReasoning**: Open Math Reasoning dataset for reinforcement learning
* **OpenCodeReasoning**: Open Code Reasoning dataset for reasoning with neural networks
* **Nemotron-CrossThink**: Nemotron-CrossThink dataset for machine learning

### Spaces

Join our community of developers and researchers by creating or joining public spaces to:

* **Host** models, datasets, and applications
* **Collaborate** on AI projects
* **Share knowledge** with the community

### Jobs

Explore job openings at Hugging Face, including roles in:

* **Data Science**
* **Machine Learning Engineering**
* **Research Assistant**

### Pricing

Get started with our platform for free. Choose from our enterprise pricing plans, starting at $20/user/month.

| Plan | Price |
| --- | --- |
| Individual | $0/hour |
| Business | $10/user/month |
| Enterprise | Custom quote |

## Explore AI Apps

Browse over 1M+ models to generate applications in:

* **Image Editing**
* **Music Generation**
* **Dialogue Systems**

Some examples of AI apps you can build with Hugging Face include:

* **Text-to-Speech**: Generate high-quality speech from text
* **Language Translation**: Translate text and speech in real-time

### Log In/Sign Up

Get started today by logging in or signing up for our platform.

| Feature | Value |
| --- | --- |
| Account Creation | Free |
| Public Model Access | Unlimited |
| Custom Spaces | Yes |

## Sign Up

Ready to join the AI community? Click here to sign up and start exploring our platform.

[Sign Up Form]

### Enterprise Hub

Get exclusive access to our enterprise-ready version of the world's leading AI platform. Subscribe for $20/user/month with your organization.

| Plan | Price |
| --- | --- |
| Basic | $10/user/month |
| Pro | $50/user/month |
| Enterprise | Custom quote |

## About Us

Learn more about Hugging Face and our mission to accelerate machine learning.

[About Page]

### Terms of Service

Read our terms of service to ensure you understand how we use your data.

[Terms Of Service]

### Privacy

View our privacy policy to learn more about how we protect your data.

[Privacy Policy]

## Press

Stay up-to-date with the latest news and updates from Hugging Face.

[Press Page]

### Resources

Discover more resources, including documentation, blogs, and a forum for the AI community.

[Resources Page]

### Social

Connect with us on social media to stay updated on our latest developments.

[Social Media Links]

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:
stream_brochure("HuggingFace", "https://huggingface.co")

## Translate to Persian

In [106]:
def get_brochure_Persian_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure in spanish of the team in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:40_000] # Truncate if more than 40,000 characters
    return user_prompt

In [102]:
def stream_brochure_Persian(company_name, url):
    stream =  ollama.chat(
      model = "llama3.2:1b",
     messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_Persian_user_prompt(company_name, url)}
      ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        # بررسی اینکه chunk اصلاً content داره یا نه
        content_piece = getattr(chunk, 'message', {}).get('content', '')
        response += content_piece
        response = response.replace("```", "").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure_Persian("HuggingFace", "https://huggingface.co")

In [103]:
!ollama pull deepseek-r1:1.5b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling aabd4debf0c8: 100% ▕▏ 1.1 GB                         
pulling 369ca498f347: 100% ▕▏  387 B                         
pulling 6e4c38e1172f: 100% ▕▏ 1.1 KB                         
pulling f4d24e9138dd: 100% ▕▏  148 B                         
pulling a85fe2a2e58e: 100% ▕▏  487 B                         
verifying sha256 digest 
writing manifest 
success 


In [ ]:
!ollama pull deepseek-v3

In [107]:
def stream_brochure_Persian1(company_name, url):
    stream =  ollama.chat(
      model = "deepseek-v3",
     messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_Persian_user_prompt(company_name, url)}
      ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        # بررسی اینکه chunk اصلاً content داره یا نه
        content_piece = getattr(chunk, 'message', {}).get('content', '')
        response += content_piece
        response = response.replace("```", "").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure_Persian1("HuggingFace", "https://huggingface.co")